In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import warnings
import plotly.express as px

In [ ]:
df = pd.read_csv('/kaggle/input/penguin-size-clutch-and-blood-isotope-data/penguins_raw.csv', index_col='Unnamed: 0')

In [ ]:
df.head()

In [ ]:
df.info()

Visualize the missing values using a heat map

In [ ]:
colours = ['#79de64', '#311d20'] 
sns.heatmap(df.isnull(), cmap=sns.color_palette(colours))

Let's calculate as a percentage how many gaps we have in the columns.

In [ ]:
for col in df.columns:
    pct_missing = np.mean(df[col].isnull())
    print('{} - {}%'.format(col, round(pct_missing*100)))

Let's take a look at exactly what values are contained in the "Comments" column

In [ ]:
df['Comments'].unique()

For ornithologists, this information is certainly important, but not for us (data scientist). Delete it.

In [ ]:
df = df.drop(['Comments'], axis=1)

In [ ]:
df.nunique()

Let's see what kinds of penguins we are dealing with.

In [ ]:
df['Species'].value_counts()

In [ ]:
colors = ["#B9DDF1", "#9FCAE6", "#73A4CA", "#497AA7", "#2E5B88"]
labels = df['Species'].value_counts().index
fig, ax = plt.subplots()
ax.pie(df['Species'].value_counts(),labels=labels, colors = colors, autopct = '%1.1f%%',
      wedgeprops = {"linewidth": 1, "edgecolor": "white"})
plt.show()

What islands do they live on?

In [ ]:
df['Island'].value_counts()

In [ ]:
colors = ["#73A4CA", "#9FCAE6","#B9DDF1", "#9FCAE6", "#73A4CA"]
labels = df['Island'].value_counts().index
fig, ax = plt.subplots()
ax.pie(df['Island'].value_counts(),labels=labels, colors = colors, autopct = '%1.1f%%',
      wedgeprops = {"linewidth": 1, "edgecolor": "white"})
plt.show()

To solve the classification problem and build a machine learning model, we leave the following set of columns.

In [ ]:
df = df[['Species', 'Island', 'Culmen Length (mm)', 'Culmen Depth (mm)', 'Flipper Length (mm)', 'Body Mass (g)', 'Sex']]

To begin with, we binarize the column with selection by gender.

In [ ]:
df['Sex'] = df['Sex'].apply(lambda x: 1 if x == 'MALE' else 0)

In [ ]:
df.sample(5)

In [ ]:
df['Sex'].value_counts().values

In [ ]:
sns.barplot(data=df, x='Species',y=df.Species.index, hue='Sex', palette="Purples")
plt.xticks(fontsize=6)
plt.show()

Let's use Boxplot to see how the body weight is distributed, depending on the type of penguins.

In [ ]:
sns.set_context(rc={"font.size":10,"axes.titlesize":8,"axes.labelsize":10})  
sns.boxplot(x=df['Species'], y=df['Body Mass (g)'], hue=df['Sex'], palette="PRGn")
plt.xticks(fontsize=6)
plt.show()

The two species have body mass values that are very different from the population averages. Let's look at them.

In [ ]:
df[df['Species']=='Chinstrap penguin (Pygoscelis antarctica)']['Body Mass (g)'].describe()

There are no anomalies in the mass, a large mass is observed in males, most likely they are adults, and too small a mass in young females. The data is fine.

In [ ]:
df[(df['Species']=='Chinstrap penguin (Pygoscelis antarctica)') & (df['Body Mass (g)']>=4500)]

In [ ]:
df[(df['Species']=='Chinstrap penguin (Pygoscelis antarctica)') & (df['Body Mass (g)']<3000)]

And where are the islands inhabited by penguins? Having received the names from our data, enriching them with geographical coordinates (latitude and longitude) from Wikipedia and visualizing them using the scatter_mapbox tool.

In [ ]:
df['Island'].unique()

In [ ]:
island = {'Location': ['Torgersen', 'Biscoe', 'Dream'], 'Latitude': [-64.766667, -65.433333, -64.733333], 'Longitude': [-64.083333,-65.5, -64.233333]} 
df_island = pd.DataFrame(island)

In [ ]:
df_island.head()

Swimming in waters where the Antarctic circumpolar current (the current of Western Winds) is perhaps one of the worst places on our planet for humans, but penguins  like it there))))

In [ ]:
fig = px.scatter_mapbox(df_island, lat = 'Latitude', lon = 'Longitude', text = 'Location',
                                                zoom = 4, mapbox_style = 'open-street-map')                      
fig.show() 

In order for our model to be able to learn, it is necessary to assign categorical values to the Species column.

In [ ]:
df['Species'] = df['Species'].astype('category')

In [ ]:
df.dtypes

In [ ]:
df['Species_category'] = df['Species'].cat.codes

We will do the same with the Island column.

In [ ]:
df['Island'] = df['Island'].astype('category')
df['Island_category'] = df['Island'].cat.codes

In [ ]:
df.sample(5)

In our dataframe, we will leave only numeric and categorical values. Let's build a correlation matrix.

In [ ]:
df = df[['Culmen Length (mm)', 'Culmen Depth (mm)', 'Flipper Length (mm)', 'Body Mass (g)', 'Sex', 'Species_category', 'Island_category']]

In [ ]:
sns.heatmap(df.corr(), annot=True, square=True, linewidths=0.5, linecolor='white', cmap='Blues')

Using the pairplot method, let's take a look at how the values are distributed, whether it is possible to visually distinguish individual groups / species

In [ ]:
sns.pairplot(df, hue='Species_category')

Adelie Penguin (Pygoscelis adeliae)          0
Chinstrap penguin (Pygoscelis antarctica)    1
Gentoo penguin (Pygoscelis papua)            2

What conclusions can be drawn:
* Gentoo penguin is the largest.
* The habitats of Chinstrap penguin and Gentoo penguin do not overlap, but Adelie Penguin live together with each of the species, neighbors on the island, so to speak.

I will not describe the differences in flippers and bills, this can be seen on the graphs.

In [ ]:
df[df['Island_category']==2]['Species_category'].value_counts()

In [ ]:
df[df['Island_category']==1]['Species_category'].value_counts() 

In [ ]:
df[df['Island_category']==0]['Species_category'].value_counts() 

Delete the missing values, only 2 lines in total.

In [ ]:
df = df.dropna()

In [ ]:
df.info()

Let's start building the model. For this classification task we will use Linear Discriminant Analysis

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from collections import Counter
from sklearn.metrics import accuracy_score

The value we will predict is Species_category

In [ ]:
y = df['Species_category']
X = df.drop('Species_category',axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state = 0)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
print(y_test.unique())
Counter(y_train)

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [ ]:
lda = LinearDiscriminantAnalysis()

In [ ]:
lda.fit(X_train, y_train)

In [ ]:
lda.predict(X_test)

The model coped with the task perfectly, accuracy is 100%.

In [ ]:
print (f"Accuracy: {accuracy_score(y_test, lda.predict(X_test))}")
print (classification_report(y_test, lda.predict(X_test)))

In [ ]:
model_matrix = confusion_matrix(y_test, lda.predict(X_test), labels = [0,1,2])

In [ ]:
model_matrix_df = pd.DataFrame(model_matrix)
model_matrix_df